# Data Collection and Processing
This notebook will collect and clean the data in order to produce a complete dataset containing player statistics with their labels of whether they made the All-star team in a specific season.

The data is obtained from a combination of the Lahman Database (https://sabr.org/lahman-database/) and from the `pybaseball` API (https://github.com/jldbc/pybaseball).

In [1]:
import pandas as pd
import numpy as np
import pybaseball

## Collecting Player Statistics
First, we begin by defining the years which we are interesting in obtaining player statistics to form the dataset. 

The dataset will include player statistics between the 2008 and 2023 (inclusive) seasons. We choose to exclude data from the 2020 season due to the COVID pandemic resulting in the cancellation of the All-star game. 

Additionally, because the goal is to build a predictor of who makes the All-star team based on mid-season statistic (prior to the All-star game), we only concern ourselves with player statistics from the start of the season to the middle of the year.

In [8]:
df_player_stats = pd.DataFrame()

In [ ]:
years = list(range(2008, 2020)) + list(range(2021, 2024))
for year in years:
    stats = pybaseball.batting_stats_range(start_dt=f'{year}-03-01', end_dt=f'{year}-06-30')
    stats['year'] = year   
    df_player_stats = pd.concat([df_player_stats, stats])

We now have a dataset that contains all mid-season player statistics. Unfortuantely, accented characters in names tend to not be displayed properly. For example, Jose Bautista's name is displayed as:

In [25]:
df_player_stats[48:49]

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID,year
50,Jos\xc3\xa9 Bautista,27,6215,Maj-NL,Pittsburgh,75,267,232,32,60,...,3,8,1,1,0.259,0.33,0.431,0.761,430832,2008


To address this we can create a dictionary mapping from an accented character to a "normalized" character.

In [26]:
utf8_to_normalized = {
    r'\xc3\x80': 'A', r'\xc3\x81': 'A', r'\xc3\x82': 'A', r'\xc3\x83': 'A', r'\xc3\x84': 'A',
    r'\xc3\x85': 'A', r'\xc3\x86': 'AE', r'\xc3\x87': 'C', r'\xc3\x88': 'E', r'\xc3\x89': 'E',
    r'\xc3\x8a': 'E', r'\xc3\x8b': 'E', r'\xc3\x8c': 'I', r'\xc3\x8d': 'I', r'\xc3\x8e': 'I',
    r'\xc3\x8f': 'I', r'\xc3\x90': 'D', r'\xc3\x91': 'N', r'\xc3\x92': 'O', r'\xc3\x93': 'O',
    r'\xc3\x94': 'O', r'\xc3\x95': 'O', r'\xc3\x96': 'O', r'\xc3\x98': 'O', r'\xc3\x99': 'U',
    r'\xc3\x9a': 'U', r'\xc3\x9b': 'U', r'\xc3\x9c': 'U', r'\xc3\x9d': 'Y', r'\xc3\x9e': 'TH',
    r'\xc3\x9f': 'ss', r'\xc3\xa0': 'a', r'\xc3\xa1': 'a', r'\xc3\xa2': 'a', r'\xc3\xa3': 'a',
    r'\xc3\xa4': 'a', r'\xc3\xa5': 'a', r'\xc3\xa6': 'ae', r'\xc3\xa7': 'c', r'\xc3\xa8': 'e',
    r'\xc3\xa9': 'e', r'\xc3\xaa': 'e', r'\xc3\xab': 'e', r'\xc3\xac': 'i', r'\xc3\xad': 'i',
    r'\xc3\xae': 'i', r'\xc3\xaf': 'i', r'\xc3\xb0': 'd', r'\xc3\xb1': 'n', r'\xc3\xb2': 'o',
    r'\xc3\xb3': 'o', r'\xc3\xb4': 'o', r'\xc3\xb5': 'o', r'\xc3\xb6': 'o', r'\xc3\xb8': 'o',
    r'\xc3\xb9': 'u', r'\xc3\xba': 'u', r'\xc3\xbb': 'u', r'\xc3\xbc': 'u', r'\xc3\xbd': 'y',
    r'\xc3\xbe': 'th', r'\xc3\xbf': 'y', r"\'": r"'"
}

In [30]:
def replace_with_dictionary(string, dictionary):
    for key, value in dictionary.items():
        string = string.replace(key, value)
    return string

In [37]:
array = df_player_stats['Name'].tolist()
for i in range(len(array)):
    array[i] = replace_with_dictionary(array[i], utf8_to_normalized)
df_player_stats['Name'] = array

Now, we can see that the accented names are now displayed in a readible fashion. 

In [78]:
df_player_stats[48:49]

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID,year
50,Jose Bautista,27,6215,Maj-NL,Pittsburgh,75,267,232,32,60,...,3,8,1,1,0.259,0.33,0.431,0.761,430832,2008


## Collecting All-star Player Data
`pybaseball` does not have any information on whether the players made it to the All-star team or not, therefore we will rely on the Lahman Database which does contain this information.

The Lahman database contains a lot of additionally information about the MLB in general, thus we only concern ourselves with specific datasheets. These are the `AllstarFull`, `People`, and `Appearences` datasheets. 

The goal here is to create a dataset that contains all All-star players to then be later be used to label our complete player statistic data on whether each player was or was not an All-star.

In [80]:
file_lahman = pd.ExcelFile('lahmanData.xlsx')

In [104]:
df_allstar = pd.read_excel(file_lahman, 'AllstarFull')
df_people = pd.read_excel(file_lahman, 'People')
df_appearances = pd.read_excel(file_lahman, 'Appearances')

Now we are merging the name and birth year information from `df_people` into `df_allstar`. We are going to be matching the information by `playerID`.

In [105]:
df_allstar = pd.merge(df_allstar, df_people[['playerID', 'nameFirst', 'nameLast', 'birthYear']], on='playerID', how='left')
df_allstar.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos,nameFirst,nameLast,birthYear
0,aaronha01,1957,0,NLS195707090,MLN,NL,1,9.0,Hank,Aaron,1934.0
1,aaronha01,1958,0,ALS195807080,MLN,NL,1,9.0,Hank,Aaron,1934.0
2,aaronha01,1959,1,NLS195907070,MLN,NL,1,9.0,Hank,Aaron,1934.0
3,aaronha01,1959,2,NLS195908030,MLN,NL,1,9.0,Hank,Aaron,1934.0
4,aaronha01,1960,1,ALS196007110,MLN,NL,1,9.0,Hank,Aaron,1934.0


Formatting for the birthyear and name of the Allstar players.

In [106]:
df_allstar['birthYear'] = df_allstar['birthYear'].astype(int)
df_allstar['Name'] = df_allstar['nameFirst'] + ' ' + df_allstar['nameLast']
df_allstar.drop(columns=['nameFirst', 'nameLast'], inplace=True)
df_allstar.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos,birthYear,Name
0,aaronha01,1957,0,NLS195707090,MLN,NL,1,9.0,1934,Hank Aaron
1,aaronha01,1958,0,ALS195807080,MLN,NL,1,9.0,1934,Hank Aaron
2,aaronha01,1959,1,NLS195907070,MLN,NL,1,9.0,1934,Hank Aaron
3,aaronha01,1959,2,NLS195908030,MLN,NL,1,9.0,1934,Hank Aaron
4,aaronha01,1960,1,ALS196007110,MLN,NL,1,9.0,1934,Hank Aaron


We have All-star information since 1934, but our `Appearences` datasheets only carries information from 1970 and onward. Therefore, we are going to be removing the All-star information prior from 1970. This, is okay since we only truly need All-star information starting from 2008.

In [107]:
df_allstar = df_allstar[df_allstar['yearID'] >= 1970]
df_allstar.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos,birthYear,Name
16,aaronha01,1970,0,NLS197007140,ATL,NL,1,9.0,1934,Hank Aaron
17,aaronha01,1971,0,ALS197107130,ATL,NL,1,9.0,1934,Hank Aaron
18,aaronha01,1972,0,NLS197207250,ATL,NL,1,9.0,1934,Hank Aaron
19,aaronha01,1973,0,ALS197307240,ATL,NL,1,3.0,1934,Hank Aaron
20,aaronha01,1974,0,NLS197407230,ATL,NL,1,9.0,1934,Hank Aaron


Now, this project will be utilizing hitting stats to make its prediciton of who makes the All-star team as a hitter. Therefore, we are not interested in predicting players who make the All-star team as a pitcher as their hitting stats will be worse than that of a All-star hitter. 

To identify which All-star players are pitchers, we are going to define an pitcher as: Any player who has pitched in more than 50% of the game appearences.

In [108]:
df_appearances['is_pitcher'] = (df_appearances['G_p'] / df_appearances['G_all']).round()
df_appearances.head()

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr,is_pitcher
0,1970,ATL,NL,aaronha01,150,135,150,136,0,0,...,0,0,0,0,125,125,0,15,0,0.0
1,1970,ATL,NL,aaronto01,44,12,44,27,0,0,...,0,0,10,0,2,12,0,17,4,0.0
2,1970,ATL,NL,asprobo01,62,28,62,35,0,0,...,30,4,1,0,0,1,0,29,0,0.0
3,1970,ATL,NL,bakerdu01,13,5,13,11,0,0,...,0,0,5,3,3,11,0,1,6,0.0
4,1970,ATL,NL,barbest01,5,2,5,5,5,0,...,0,0,0,0,0,0,0,0,0,1.0


Now that we have identified whether a player is considered a pitcher or not for a given season. We can bring this information over into our All-star data, and then remove all All-star players who are considered pitchers.

In [109]:
df_allstar = df_allstar.merge(df_appearances[['playerID', 'yearID', 'is_pitcher']], on=['playerID', 'yearID'], how='left')

In [114]:
df_allstar[20:30]

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos,birthYear,Name,is_pitcher
20,aguilri01,1993,0,ALS199307130,MIN,AL,1,NaN,1961,Rick Aguilera,1.0
21,albieoz01,2018,0,NLS201807170,ATL,NL,1,NaN,1997,Ozzie Albies,0.0
22,albieoz01,2021,0,NLS202107130,ATL,NL,1,NaN,1997,Ozzie Albies,0.0
23,albieoz01,2023,0,ALS202307110,ATL,NL,1,NaN,1997,Ozzie Albies,0.0
24,alcansa01,2019,0,ALS201907090,MIA,NL,1,NaN,1995,Sandy Alcantara,1.0
25,alcansa01,2022,0,NLS202207190,MIA,NL,1,NaN,1995,Sandy Alcantara,1.0
26,alcansa01,2022,0,NLS202207190,MIA,NL,1,NaN,1995,Sandy Alcantara,1.0
27,alexado01,1988,0,NLS198807120,DET,AL,0,NaN,1950,Doyle Alexander,1.0
28,alfoned01,2000,0,NLS200007110,NYN,NL,1,NaN,1973,Edgardo Alfonzo,0.0
29,allendi01,1970,0,NLS197007140,SLN,NL,1,3.0,1942,Dick Allen,0.0


In [118]:
df_allstar = df_allstar[df_allstar['is_pitcher'] != 1]
df_allstar.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos,birthYear,Name,is_pitcher
0,aaronha01,1970,0,NLS197007140,ATL,NL,1,9.0,1934,Hank Aaron,0.0
1,aaronha01,1971,0,ALS197107130,ATL,NL,1,9.0,1934,Hank Aaron,0.0
2,aaronha01,1972,0,NLS197207250,ATL,NL,1,9.0,1934,Hank Aaron,0.0
3,aaronha01,1973,0,ALS197307240,ATL,NL,1,3.0,1934,Hank Aaron,0.0
4,aaronha01,1974,0,NLS197407230,ATL,NL,1,9.0,1934,Hank Aaron,0.0


## Labeling Player Statistic Data
Next, we need to label our `df_player_stats` on whether they are an All-star or not. However, unlike the Lahman database which has playerID and season (year) to properly identify the players, `pybaseball` just has them labeled with their name, birthdate, and season (year) they played. Luckily, our `df_allstar` has all of this information and all we need to do is identify a player in `df_player_stats` by their unique features (full name, year played, and birth year).

However, utilizing the names of the players to match between the two datasets will not work exactly, because although the special characters has been addressed, `pybaseball` does not. include suffixes like Jr. Sr. or II, etc. Therefore, we need to again clean the names by stripping them of the suffixes. Naturally, this will lead to Jr's to have the same name as their Sr., but because we are also identifying players by the season they played and their birthyear we can still identify them apart.

In [ ]:
import re

def clean_name(name):
    name = re.sub(r'\s+(Jr\.|Sr\.|II|III|IV)$', '', name.strip(), flags=re.IGNORECASE)
    return name.lower()

In [161]:
# Apply cleaning to both DataFrames
df_player_stats['Name_clean'] = df_player_stats['Name'].apply(clean_name)
df_allstar['Name_clean'] = df_allstar['Name'].apply(clean_name)

In [162]:
all_star_records = set(zip(df_allstar['Name_clean'], df_allstar['yearID'], df_allstar['birthYear']))

In [163]:
def is_all_star(row):
    est_birth = row['year'] - row['Age']
    name_clean = row['Name_clean']
    for offset in [-2, -1, 0, 1, 2]:
        if (name_clean, row['year'], est_birth + offset) in all_star_records:
            return 1
    return 0

In [164]:
df_player_stats['All Star'] = df_player_stats.apply(is_all_star, axis=1)

In [165]:
df_player_stats[55:60]

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,...,SB,CS,BA,OBP,SLG,OPS,mlbID,year,All Star,Name_clean
58,Adrian Beltre,29,6215,Maj-AL,Seattle,79,334,299,40,74,...,7,1,0.247,0.320,0.441,0.762,134181,2008,0,adrian beltre
59,Gary Bennett,36,6260,Maj-NL,Los Angeles,9,23,21,1,4,...,0,0,0.190,0.261,0.381,0.642,110870,2008,0,gary bennett
60,Jeff Bennett,28,6234,Maj-NL,Atlanta,7,9,7,0,1,...,0,0,0.143,0.250,0.143,0.393,430831,2008,0,jeff bennett
61,Jason Bergmann,26,6216,Maj-NL,Washington,9,19,19,0,0,...,0,0,0.000,0.000,0.000,0.000,456589,2008,0,jason bergmann
62,Lance Berkman,32,6215,Maj-NL,Houston,81,346,296,72,108,...,12,2,0.365,0.448,0.699,1.147,204020,2008,1,lance berkman


We now have a complete dataset from 2008 to 2023 of player statistics of the first half of a respective season, and each player labeled if whether or not they made the All-star team that season.

## Saving Dataset
Saving the data as both a `.csv` and a `.pkl` file.

In [170]:
df_player_stats.to_pickle("player_stats.pkl")

In [171]:
df_player_stats.to_csv("player_stats.csv", index=False)